In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col
from pymongo import MongoClient
import time
import subprocess

In [2]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName('myapp')
conf.set('spark.python.worker.timeout', '600')
sc = SparkContext(conf=conf)

In [3]:
from pyspark.ml import PipelineModel
from pyspark.ml.classification import NaiveBayes

model_path = 'C:/Users/hanane/Desktop/BigDataProje/apacheflume/ModelSave/'
pipeline_model = PipelineModel.load(model_path)

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession(sc)

In [5]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col


data = spark.read.text("hdfs://localhost:9000/user/Hadoop/twit_test/*")

filtered_data = data.filter(~data["value"].startswith('   ') )
filtered_data.show()


split_col = split(filtered_data['value'], ',')

filtered_data = filtered_data.withColumn('id', split_col.getItem(0)) \
                             .withColumn('topic', split_col.getItem(1)) \
                             .withColumn('sentiment', split_col.getItem(2)) \
                             .withColumn('content', split_col.getItem(3))

filtered_data = filtered_data.drop('value')
filtered_data = filtered_data.na.drop()
predictions = pipeline_model.transform(filtered_data)
predictions.select("id", "topic", "content", "prediction").show(truncate=False)

+--------------------+
|               value|
+--------------------+
|Only on @WatchMix...|
|11028,TomClancysG...|
|                    |
|store.playstation...|
|2695,Borderlands,...|
|10719,RedDeadRede...|
|2703,Borderlands,...|
|4334,CS-GO,Neutra...|
|10865,TomClancysG...|
|12443,WorldOfCraf...|
|10007,PlayerUnkno...|
|6854,johnson&john...|
|4811,GrandTheftAu...|
|7806,MaddenNFL,Po...|
|11674,Verizon,Irr...|
|12714,WorldOfCraf...|
|5048,GrandTheftAu...|
|5025,GrandTheftAu...|
|8728,NBA2K,Negati...|
|11824,Verizon,Neu...|
+--------------------+
only showing top 20 rows

+-----+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|id   |topic                            |content                                 

In [ ]:
from pymongo import MongoClient
import time

documents = predictions.select("id", "topic", "content", "prediction").collect()
documents = [row.asDict() for row in documents]

client = MongoClient('mongodb://localhost:27017')
db = client.BigData
collection = db.new

for document in documents:
    collection.insert_one(document)
    print(f"Inserted document: {document['id']}")
    time.sleep(6)


Inserted document: 11028
Inserted document: 2695
Inserted document: 10719
Inserted document: 2703
Inserted document: 4334
Inserted document: 10865
Inserted document: 12443
Inserted document: 10007
Inserted document: 6854
Inserted document: 4811
Inserted document: 7806
Inserted document: 11674
Inserted document: 12714
